In [82]:
import numpy as np

In [116]:
def determinant(matrix):
    if matrix.shape == (2, 2):
        return matrix[0, 0] * matrix[1, 1] - matrix[0, 1] * matrix[1, 0]
    elif matrix.shape == (3, 3):
        return (matrix[0, 0] * (matrix[1, 1] * matrix[2, 2] - matrix[1, 2] * matrix[2, 1])
                - matrix[0, 1] * (matrix[1, 0] * matrix[2, 2] - matrix[1, 2] * matrix[2, 0])
                + matrix[0, 2] * (matrix[1, 0] * matrix[2, 1] - matrix[1, 1] * matrix[2, 0]))
    else:
        raise ValueError("Sorry, only 2x2 and 3x3 matrix's sizesupported")

In [117]:
def inverse_matrix(matrix):
    """We put in this function our array[][] (matrix) and return also matrix"""

    # Gauss Method
    size = matrix.shape[0]  # predict that we are working with the quadratic matrix and function size will return the number of rows in this case/ shape[1] return the number of columns
    # print(size)  # I have checked it

    # Here we checked if det = 0, then we reject request/ Otherwise all okay/ if it's the right size of the matrix
    det = determinant(matrix)
    if abs(det) < 0:
        print("Determinant of this matrix equals or close to zero! So, we can't find the inverse matrix")
        return None

    """
        [*][*][*]/[1][0][0]
        [*][*][*]/[0][1][0] <- augmented matrix (columns: size * 2; rows: size)
        [*][*][*]/[0][0][1]
    """
    augmented_matrix = np.zeros((size, 2 * size))

    # we need to fill matrix on the left side with our matrix and the right side with identity matrix
    for i in range(size):
        for j in range(size):
            augmented_matrix[i, j] = matrix[i, j]  # our left part with given matrix
        augmented_matrix[i, i + size] = 1.0  # our right part with identity matrix
    # and so our augmented matrix will be 3 by 6 (rows: 3, columns: 6)

    # check that everything is alright
    # print("Augmented Matrix")
    # for row in augmented_matrix:
    #     print(", ".join(map(str, row)))
    # print()

    # Direct of the Gaussian method
    for i in range(size):

        # Diagonal element should be equal to "1", so we need to divide by this element
        diag_element = augmented_matrix[i, i]

        for j in range(2 * size):  # because this is our matrix + identity matrix we will use "2 * size"/ our matrix is quadratic
            augmented_matrix[i, j] /= diag_element  # every diagonal element must be 1

        # Reset the elements in the rows in the current column to zero
        for k in range(size):
            if k != i:
                factor = augmented_matrix[k, i]  # not a diagonal number
                for j in range(2 * size):  # like in the previous loop
                    augmented_matrix[k, j] -= factor * augmented_matrix[i, j]  # "nullarize" or reset value for arr[k][i]
                # it's easy to see this, but in reality it's more complex
        # Example:
        # [2][1]
        # [5][3]
        # =>
        # [1][0.5]
        # [5][3]
        # =>
        # 2nd row = 2nd row - 5 * 1st row
        # =>
        # [1][0.5]
        # [0][0.5]
    # This example represents step-by-step dividing a row to zero value

    # Our inverse matrix is on the right side <= or => our changed identity matrix
    inverse_matrix = np.zeros((size, size))  # initialize future answer like arr[][]
    for i in range(size):
        for j in range(size):
            inverse_matrix[i, j] = augmented_matrix[i, j + size]

    """
        *, *, *, -0.26086956521739113, -0.4130434782608696, 0.7173913043478262
        *, *, *, 0.47826086956521735, 0.17391304347826086, -0.5652173913043479
        *, *, *, -0.2608695652173913, 0.08695652173913045, 0.2173913043478261
                 |            This is our inverse matrix                      |
    """

    return inverse_matrix

In [118]:
def transpose(matrix):
    return np.transpose(matrix) # 11 -> 11; 12 -> 21; 13 -> 31; and so on

In [119]:
def matrix_multiply(A, B):
    return np.dot(A, B) # every row of first matrix multiply by every column of second

In [120]:
def tikhonov_regularization(A, y, L):
    AT = transpose(A)
    AT_A = matrix_multiply(AT, A)

    # add regularization parameter lambdaI
    AT_A += L * np.eye(AT_A.shape[0])  # lambdaI, like in this code we creating an identity matrix with size like AT_A matrix

    # (A^T * A + λI)^(-1)
    AT_A_inv = inverse_matrix(AT_A)

    # (A^T * A + λI)^(-1) * A^T * y
    AT_y = matrix_multiply(AT, y)
    return matrix_multiply(AT_A_inv, AT_y)

In [121]:
A = np.array([[1.0, 2.0],
              [2.0, 4.001]])

In [122]:
y = np.array([3.0, 6.001])

In [123]:
lambdas = [1000, 100, 10, 1, 0.1, 0.01, 0.001, 0.0001, 0.00001, 0.0000000000000000001]

In [124]:
A_inv = inverse_matrix(A)
if A_inv is not None:
    direct_solution = matrix_multiply(A_inv, y)
    print("Direct solution:")
    print(direct_solution)
    print()

# Решение с Тихоновской регуляризацией для различных значений λ
for lambd in lambdas:
    reg_solution = tikhonov_regularization(A, y, lambd)
    print(f"Tichonov regularization solution, lambda = {lambd}:")
    print(reg_solution)
    print()

Direct solution:
[1. 1.]

Tichonov regularization solution, lambda = 1000:
[0.01463598 0.02927782]

Tichonov regularization solution, lambda = 100:
[0.12000832 0.24006464]

Tichonov regularization solution, lambda = 10:
[0.42853061 0.85723264]

Tichonov regularization solution, lambda = 1:
[0.57682251 1.15387572]

Tichonov regularization solution, lambda = 0.1:
[0.59749894 1.19523649]

Tichonov regularization solution, lambda = 0.01:
[0.59964983 1.19953553]

Tichonov regularization solution, lambda = 0.001:
[0.59988002 1.19996001]

Tichonov regularization solution, lambda = 0.0001:
[0.60004554 1.19993124]

Tichonov regularization solution, lambda = 1e-05:
[0.60148134 1.19921889]

Tichonov regularization solution, lambda = 1e-19:
[1.00000002 0.99999997]



In [125]:
'''
1) As lambda increases:
 a) The solution becomes more stable when the determinant equal to zero or close to. Such a matrix (when the determinant equal to zero) cannot be used to solve equations using standard methods, since the system of linear equations becomes indeterminate.
 b) Small errors or noise in the data do not lead to large disturbances as a result. This can be useful, when the determinant is close to zero.
2) As lambda decreases:
 a) The regularization becomes unstable and the solution approaches the direct solution.
 b) The solution becomes more unpredictable and unreliable, for poorly conditioned systems.
'''

'\n1) As lambda increases:\n a) The solution becomes more stable when the determinant equal to zero or close to. Such a matrix (when the determinant equal to zero) cannot be used to solve equations using standard methods, since the system of linear equations becomes indeterminate.\n b) Small errors or noise in the data do not lead to large disturbances as a result. This can be useful, when the determinant is close to zero.\n2) As lambda decreases:\n a) The regularization becomes unstable and the solution approaches the direct solution.\n b) The solution becomes more unpredictable and unreliable, for poorly conditioned systems.\n'

In [126]:
for lambd in lambdas:

    reg_solution = tikhonov_regularization(A, y, lambd)

    print(f"λ = {lambd}")
    if A_inv is not None:
        direct_solution = matrix_multiply(A_inv, y)
        diff = np.linalg.norm(direct_solution - reg_solution)
        print(f"Deviation from the direct solution: {diff}")

    print()
    #For small values of lambda, the result becomes closer to a direct solution

λ = 1000
Deviation from the direct solution: 1.383200561652171

λ = 100
Deviation from the direct solution: 1.1627068021243416

λ = 10
Deviation from the direct solution: 0.5890329214366938

λ = 1
Deviation from the direct solution: 0.4502853813393582

λ = 0.1
Deviation from the direct solution: 0.44735264606217384

λ = 0.01
Deviation from the direct solution: 0.44731944518667854

λ = 0.001
Deviation from the direct solution: 0.4473030292723838

λ = 0.0001
Deviation from the direct solution: 0.44714211284883293

λ = 1e-05
Deviation from the direct solution: 0.4455393251612726

λ = 1e-19
Deviation from the direct solution: 3.469986348575745e-08

